In [1]:
import os
import yaml

def get_files(path):
    entries = {}
    with os.scandir(path) as it:
        for entry in it:
            name = entry.name
            name_ext = name.split('.')

            # Skip if the entry is a hidden entry (.*) or a jupyter-book entry (_*)
            if name.startswith('.') or name.startswith('_'):
                continue
                
            if entry.is_file():
                if len(name_ext) > 1 and name_ext[-1] == 'ipynb':
                    entries[name_ext[0]] = None
            else:
                entries[name_ext[0]] = get_files(os.path.join(path, name))
            
    return entries

def files_to_yaml(files, dirname):
    sub_yaml = []
    for key, value in files.items():
        if key.split('.')[0] == 'index':
            continue
        new_dirname = os.path.join(dirname, key)

        entry = {}
        if value is not None:
            sections = files_to_yaml(value, new_dirname)
            if len(sections) == 0:
                continue
            entry['sections'] = sections
        entry['file'] = new_dirname if value is None else os.path.join(new_dirname, 'index')
        entry['title'] = key

        sub_yaml.append(entry)
        
    return sub_yaml

In [2]:
files = get_files('.')

toc_yaml = {
    'format': 'jb-book',
    'root': 'root',
    'parts': [],
}

for key, value in files.items():
    if value is None:
        continue
        
    entry = {}
    entry['caption'] = key
    entry['chapters'] = files_to_yaml(value, key)
    toc_yaml['parts'].append(entry)
    
with open('_toc.yml', 'w') as file:
    yaml.dump(toc_yaml, file)

In [3]:
import glob
import nbformat

def change_metadata(path):
    notebook = nbformat.read(path, nbformat.NO_CONVERT)

    changed = False
    for cell in notebook.cells:
        metadata = cell.get('metadata', {})
        tags = set(metadata.get('tags', []))
        jupyter = metadata.get('jupyter', {})

        hiddens = [jupyter.get('source_hidden', False), jupyter.get('output_hidden', False)]
        new_tags = tags - {'hide-cell', 'hide-input', 'hide-output', 'remove-cell', 'remove-input', 'remove-output'}
        if all(hiddens):
            new_tags.add('remove-cell')
        elif hiddens[0]:
            new_tags.add('remove-input')
        elif hiddens[1]:
            new_tags.add('remove-output')

        if new_tags != tags:
            cell['metadata']['tags'] = list(new_tags)
            changed = True

    if changed:
        nbformat.write(notebook, path)

In [4]:
for key, value in files.items():
    if value is None:
        continue 
        
    notebook_paths = glob.glob(f'./{key}/**/*.ipynb', recursive=True)
    for notebook_path in notebook_paths:
        change_metadata(notebook_path)

In [5]:
!rm -r _build
!jupyter-book config sphinx .

settings = ['suppress_warnings = ["myst.header"]', 'myst_dmath_double_inline = True']
with open('conf.py', 'a') as file:
    file.write('\n# Belows are more customized settings')
    for setting in settings:
        file.write(f'\n{setting}')
    
!sphinx-build . ./_build/html/ -b html
!rm conf.py

Wrote conf.py to /home/l1qiao/Projects/joeyonng-notebook
Running Sphinx v4.3.0
making output directory... done
[etoc] Changing master_doc to 'root'
[etoc] Excluded 2 extra file(s) not in toc
checking for /home/l1qiao/Projects/joeyonng-notebook/references.bib in bibtex cache... not found
parsing bibtex file /home/l1qiao/Projects/joeyonng-notebook/references.bib... parsed 5 entries
myst v0.16.1: MdParserConfig(renderer='sphinx', commonmark_only=False, enable_extensions=['colon_fence', 'dollarmath', 'linkify', 'substitution', 'tasklist'], linkify_fuzzy_links=True, dmath_allow_labels=True, dmath_allow_space=True, dmath_allow_digits=True, dmath_double_inline=True, update_mathjax=True, mathjax_classes='tex2jax_process|mathjax_process|math|output_area', disable_syntax=[], url_schemes=['mailto', 'http', 'https'], heading_anchors=None, heading_slug_func=None, html_meta=[], footnote_transition=True, substitutions=[], sub_delimiters=['{', '}'], words_per_minute=200)
building [mo]: targets for 0 p